In [1]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_MP\Log\data_loader.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_MP\Data\Enginered_Data\feature_enginering.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [2]:
df.head()

,Unnamed: 0,Video ID,Video Title,Channel,Published Date,Views,Likes,Comments,Subscribers,Channel Views,...,Category_Cluster_y,Year,Month,Day,DayOfWeek,DayName,Views_per_Sub,Engagement,Like_per_Sub,Comment_per_Sub
0,0,-7yvyIvJXH0,O RELÓGIO SECRETO DO GTA 5!,GamerCosta,2025-10-20 00:05:43+00:00,34727,686,16,4190000,1909505035,...,2,2025,10,20,0,Monday,0.008288,0.020214,0.000164,0.000004
1,1,9h5Uv27uTwU,"GIRLS TRIP 🌴✨ Traumurlaub, Yoga & Surfen! 🥰",Paula Doeringer,2025-10-16 16:03:51+00:00,45151,1514,21,188000,135785655,...,2,2025,10,16,3,Thursday,0.240164,0.033996,0.008053,0.000112
2,2,SmF8VTWdqZU,Imran Riaz News About TLP Saad & Anas Rizvi #r...,Real Muslim 100,2025-10-17 03:00:36+00:00,353713,6485,127,37300,54921887,...,2,2025,10,17,4,Friday,9.482668,0.018693,0.173856,0.003405
3,3,rvn2CD4h_ac,เป็นคลิปแกล้งที่ทำให้เขาเป็น Viral เลย #kornzo,Kornzo,2025-10-18 09:07:36+00:00,2171652,78431,183,43800,88085601,...,1,2025,10,18,5,Saturday,49.579964,0.036200,1.790621,0.004178
4,4,Efr59jKIkgU,โกงยันแอดมิน กลุ่มซื้อ-ขาย Valorant [ โกงมั้ยค...,DOM,2025-10-16 10:48:06+00:00,246429,9321,949,2490000,501089468,...,0,2025,10,16,3,Thursday,0.098967,0.041675,0.003743,0.000381


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8493 entries, 0 to 8492
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          8493 non-null   int64  
 1   Video ID            8493 non-null   object 
 2   Video Title         8493 non-null   object 
 3   Channel             8492 non-null   object 
 4   Published Date      8493 non-null   object 
 5   Views               8493 non-null   int64  
 6   Likes               8493 non-null   int64  
 7   Comments            8493 non-null   int64  
 8   Subscribers         8493 non-null   int64  
 9   Channel Views       8493 non-null   int64  
 10  Country             8493 non-null   object 
 11  Region              8493 non-null   object 
 12  CategoryID          8493 non-null   int64  
 13  CategoryName        8493 non-null   object 
 14  Cluster             8493 non-null   int64  
 15  CategoryID_enc      8493 non-null   int64  
 16  Countr

In [5]:
import regex as re

non_latin_pattern = re.compile(r'\P{Latin}')

def has_non_latin(s):
    if not isinstance(s, str):
        return False
    return bool(non_latin_pattern.search(s))

# faqat object turidagi ustunlarni ajratamiz
obj_cols = df.select_dtypes(include=['object']).columns

# har bir satrda qaysi ustunlarda non-latin borligini aniqlaymiz
mask = df[obj_cols].applymap(has_non_latin).any(axis=1)
notanish_df = df[mask]

print(f"Notanish yozuvli satrlar soni: {mask.sum()}")
notanish_df.to_csv("nonlatin_rows.csv", index=False)


Notanish yozuvli satrlar soni: 8493


C:\Users\Rasulbek907\AppData\Local\Temp\ipykernel_12452\598365124.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  mask = df[obj_cols].applymap(has_non_latin).any(axis=1)


In [6]:
summary = df.select_dtypes(include=['object']).apply(
    lambda col: col.apply(has_non_latin).sum()
)
print(summary)


Video ID          7612
Video Title       8462
Channel           6627
Published Date    8493
Country           1649
Region               0
CategoryName      4714
Anomaly              0
DayName              0
dtype: int64


In [7]:
df["CategoryName"].unique()

array(['Gaming', 'People & Blogs', 'News & Politics', 'Entertainment',
       'Science & Technology', 'Howto & Style', 'Autos & Vehicles',
       'Music', 'Film & Animation', 'Sports', 'Comedy'], dtype=object)

In [8]:
category_map = {
    'Gaming': 0,
    'People & Blogs': 1,
    'News & Politics': 2,
    'Entertainment': 3,
    'Science & Technology': 4,
    'Howto & Style': 5,
    'Autos & Vehicles': 6,
    'Music': 7,
    'Film & Animation': 8,
    'Sports': 9,
    'Comedy': 10
}

df['CategoryName'] = df['CategoryName'].map(category_map)


In [9]:
summary = df.select_dtypes(include=['object']).apply(
    lambda col: col.apply(has_non_latin).sum()
)
print(summary)

Video ID          7612
Video Title       8462
Channel           6627
Published Date    8493
Country           1649
Region               0
Anomaly              0
DayName              0
dtype: int64


In [12]:
from unidecode import unidecode

# notanish yozuvli ustunlar ro'yxati
cols_to_clean = ['Video ID', 'Video Title', 'Channel', 'Published Date', 'Country']

# har bir ustun bo'yicha unidecode qo'llash
for col in cols_to_clean:
    df[col] = df[col].astype(str).apply(unidecode)

print("✅ Unidecode barcha belgilar lotin yozuviga o‘tkazildi!")


✅ Unidecode barcha belgilar lotin yozuviga o‘tkazildi!


In [15]:
df.head()

,Unnamed: 0,Video ID,Video Title,Channel,Published Date,Views,Likes,Comments,Subscribers,Channel Views,...,Category_Cluster_y,Year,Month,Day,DayOfWeek,DayName,Views_per_Sub,Engagement,Like_per_Sub,Comment_per_Sub
0,0,-7yvyIvJXH0,O RELOGIO SECRETO DO GTA 5!,GamerCosta,2025-10-20 00:05:43+00:00,34727,686,16,4190000,1909505035,...,2,2025,10,20,0,Monday,0.008288,0.020214,0.000164,0.000004
1,1,9h5Uv27uTwU,"GIRLS TRIP Traumurlaub, Yoga & Surfen!",Paula Doeringer,2025-10-16 16:03:51+00:00,45151,1514,21,188000,135785655,...,2,2025,10,16,3,Thursday,0.240164,0.033996,0.008053,0.000112
2,2,SmF8VTWdqZU,Imran Riaz News About TLP Saad & Anas Rizvi #r...,Real Muslim 100,2025-10-17 03:00:36+00:00,353713,6485,127,37300,54921887,...,2,2025,10,17,4,Friday,9.482668,0.018693,0.173856,0.003405
3,3,rvn2CD4h_ac,epnkhlipaeklngthiithamaihekhaaepn Viral ely #k...,Kornzo,2025-10-18 09:07:36+00:00,2171652,78431,183,43800,88085601,...,1,2025,10,18,5,Saturday,49.579964,0.036200,1.790621,0.004178
4,4,Efr59jKIkgU,okngyanae`dmin klumchuue`-khaay Valorant [ okn...,DOM,2025-10-16 10:48:06+00:00,246429,9321,949,2490000,501089468,...,0,2025,10,16,3,Thursday,0.098967,0.041675,0.003743,0.000381


In [16]:
import regex as re

# faqat lotin harflar, raqamlar, bo‘sh joy va oddiy belgilarni ruxsat beramiz
normal_pattern = re.compile(r'^[\p{Latin}\d\s.,!?;:\-_/()\'"&%]*$')

def has_unusual_chars(s):
    if not isinstance(s, str):
        return False
    return not bool(normal_pattern.match(s))  # agar mos kelmasa → noodatiy belgi bor

# tekshiriladigan ustunlar
cols_to_check = ['Video ID', 'Video Title', 'Channel', 'Published Date', 'Country']

# har bir ustun bo‘yicha noodatiy belgilar sonini hisoblash
for col in cols_to_check:
    count = df[col].apply(has_unusual_chars).sum()
    print(f"{col:20s} → {count} ta noodatiy qiymat")


Video ID             → 0 ta noodatiy qiymat
Video Title          → 4240 ta noodatiy qiymat
Channel              → 354 ta noodatiy qiymat
Published Date       → 8493 ta noodatiy qiymat
Country              → 0 ta noodatiy qiymat


# Video Title va Channel ustunlaridagi noodatiy belgilarni tozalash

In [17]:
import re

def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^\x00-\x7F]+', '', text)  # faqat ASCII qoldiradi
    return text

df["Video Title"] = df["Video Title"].apply(clean_text)
df["Channel"] = df["Channel"].apply(clean_text)


# Published Date ustunini datetime formatga o‘tkazish

In [18]:
df["Published Date"] = pd.to_datetime(df["Published Date"], errors='coerce')


In [20]:
import re
import unicodedata

def deep_clean_text(text):
    if not isinstance(text, str):
        return text

    # 1. Normalize (unicode form)
    text = unicodedata.normalize('NFKC', text)

    # 2. Remove control characters va ko‘rinmas belgilar
    text = re.sub(r'[\u200b-\u200f\u202a-\u202e\u2060-\u206f\u00ad\uFEFF]', '', text)

    # 3. Remove HTML entities
    text = re.sub(r'&[a-z]+;', '', text)

    # 4. Remove all non-printable characters
    text = ''.join(ch for ch in text if ch.isprintable())

    # 5. Remove remaining non-ASCII characters (optional)
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # 6. Strip whitespaces
    text = text.strip()

    return text

df["Video Title"] = df["Video Title"].apply(deep_clean_text)
df["Channel"] = df["Channel"].apply(deep_clean_text)


In [21]:
def count_unusual(text):
    if not isinstance(text, str):
        return False
    # Agar matnda ASCII’dan tashqari belgilar bo‘lsa, True
    return bool(re.search(r'[^\x00-\x7F]', text))

for col in ["Video Title", "Channel"]:
    bad_count = df[col].apply(count_unusual).sum()
    print(f"{col:20} → {bad_count} ta noodatiy qiymat")


Video Title          → 0 ta noodatiy qiymat
Channel              → 0 ta noodatiy qiymat


# DF dagi noodatiy belgilarni tekshirish 

In [24]:
import re

def count_unusual_chars(df):
    pattern = re.compile(r'[^\x00-\x7F]')  # ASCII bo'lmagan belgilarni topadi
    result = {}

    for col in df.select_dtypes(include=['object']).columns:
        # Har bir ustun uchun noodatiy belgilar bor-yo‘qligini tekshirish
        bad_count = df[col].astype(str).apply(lambda x: bool(pattern.search(x))).sum()
        result[col] = bad_count

    return result


unusual_counts = count_unusual_chars(df)

for col, count in unusual_counts.items():
    print(f"{col:20} → {count} ta noodatiy qiymat")


Video ID             → 0 ta noodatiy qiymat
Video Title          → 0 ta noodatiy qiymat
Channel              → 0 ta noodatiy qiymat
Country              → 0 ta noodatiy qiymat
Region               → 0 ta noodatiy qiymat
Anomaly              → 0 ta noodatiy qiymat
DayName              → 0 ta noodatiy qiymat


In [33]:
import sys
import os

# Source papkasini yo'lga qo'shamiz
source_path = os.path.abspath("../Source")
if source_path not in sys.path:
    sys.path.append(source_path)

print("Python executable:", sys.executable)
print("Source path added:", source_path)

# Endi modulni import qilamiz
from preprosessing import Cleaner, Encoder, Scaler

Python executable: c:\Users\Rasulbek907\Desktop\Project_MP\mpvenv\Scripts\python.exe
Source path added: c:\Users\Rasulbek907\Desktop\Project_MP\Source


In [34]:
cleaner = Cleaner(df)
df = cleaner.tozala().get_df()

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8493 entries, 0 to 8492
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   Unnamed: 0          8493 non-null   int64              
 1   Video ID            8493 non-null   object             
 2   Video Title         8493 non-null   object             
 3   Channel             8493 non-null   object             
 4   Published Date      8493 non-null   datetime64[ns, UTC]
 5   Views               8493 non-null   int64              
 6   Likes               8493 non-null   int64              
 7   Comments            8493 non-null   int64              
 8   Subscribers         8493 non-null   int64              
 9   Channel Views       8493 non-null   int64              
 10  Country             8493 non-null   object             
 11  Region              8493 non-null   object             
 12  CategoryID          8493 non-null 

In [37]:
encoder = Encoder(df, target_col="CategoryName")
encoded_df = encoder.encodla().get_df()

TypeError: Encoder.__init__() got an unexpected keyword argument 'target_col'

In [ ]:

df.info()

In [ ]:
scaler = Scaler(df)
df = scaler.scaling_qil().get_df()

In [ ]:

df.head()

In [ ]:

df.to_csv(r"C:\Users\Rasulbek907\Desktop\Final_Project\Data\Preprosessed_data\Preprosessed.csv")